In [13]:
from stravalib.client import Client

client = Client()

authorize_url = client.authorization_url(client_id=29653, redirect_uri='ec2-18-236-253-110.us-west-2.compute.amazonaws.com')
authorize_url

code = raw_input("Enter Access Token: ") 

token_response = client.exchange_code_for_token(client_id=29653, client_secret='24ea02bbb8a6ea5e1355293bb9fddb90117baf4e', code=code)
access_token = str(token_response.encode('utf-8'))

client.access_token = access_token
athlete = client.get_athlete()
# print("For {id}, I now have an access token {token}".format(id=athlete.id, token=client.access_token))
# print(client.get_athlete())


Enter Access Token: aba81be06f2eaab2d9fb77fedcf12fa18137bbab


In [14]:
name =  client.get_athlete().firstname + " " + client.get_athlete().lastname
city = client.get_athlete().city
print name
print city

Devansh Manu
Hyderabadi


In [15]:
all_activites = list(client.get_activities())


In [16]:
len(all_activites)
# for activity in all_activites:
#     print activity.map.summary_polyline
# (all_activites[9].map.summary_polyline)

30

In [17]:
all_polylines = [activity.map.summary_polyline for activity in all_activites if activity.map.summary_polyline]


In [18]:
len(all_polylines)

30

In [19]:
LATLONG = []

import polyline
for item in all_polylines:
    coords = polyline.decode(str(item))
    for x in coords:
        LATLONG.append(x)

In [20]:
import os
import folium
import operator

print(folium.__version__)

def dict_sort_des(diction):
    sorted_d = sorted(diction.items(), key=operator.itemgetter(1),reverse=True)
    return sorted_d

0.6.0


In [21]:
data = []
for i in LATLONG:
    data.append( list(i))

In [22]:
data
frequency_dict = {}
for k in LATLONG:
    l = round(k[0], 3),round(k[1], 3)
    if l in frequency_dict.keys():
        frequency_dict[l] += 1
    else:
        frequency_dict[l] = 1
address = dict_sort_des(frequency_dict)[:5][0][0]
#List fo frequent locations

In [23]:
from folium.plugins import HeatMap

m = folium.Map(address, tiles='openstreetmap', zoom_start=13)

HeatMap(data).add_to(m)

folium.plugins.HeatMap(data, radius = 20, min_opacity = 0.1, max_val = 40,gradient={.6: 'blue', .98: 'lime', 1: 'red'}).add_to(m)

m.save(os.path.join(str(name)+'heatmap.html'))

m


In [24]:
map = folium.Map(location=address, zoom_start=20)
folium.Marker(address).add_to(map)
map.save(os.path.join(str(name)+'address.html'))
map


In [25]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="Strava")
location = geolocator.reverse(address)
print(location.address)

International Institute of Information Technology, Hyderabad, Campus Road, Ward 105 Gachibowli, Greater Hyderabad Municipal Corporation West Zone, Hyderabad, Rangareddy District, Telangana, 500032, India


In [26]:
startCoords = []
endCoords = []

temp1 = [list(polyline.decode(str(i))[0]) for i in all_polylines]
temp2 = [list(polyline.decode(str(i))[-1]) for i in all_polylines]


for i in temp1:
    startCoords.append([round(i[0],3),round(i[1],3)])

    
    
for i in temp2:
    endCoords.append([round(i[0],3),round(i[1],3)])


In [27]:
from folium.plugins import MarkerCluster


m = folium.Map(location=address, zoom_start=12)

marker_cluster = MarkerCluster().add_to(m)

for start in startCoords:
    folium.Marker(
        location=start,
        popup='Start',
        icon=folium.Icon(color='green', icon='ok-sign'),
    ).add_to(marker_cluster)

for end in endCoords:
    folium.Marker(
        location=end,
        popup='End',
        icon=folium.Icon(color='red', icon='remove-sign'),
    ).add_to(marker_cluster)

folium.Marker(
        location=address,
        popup='Predicted address',
        icon=folium.Icon(color='blue'),
    ).add_to(marker_cluster)


m.save(os.path.join(str(name)+'proof.html'))
m